<a href="https://colab.research.google.com/github/amymargo/Rotten-Tomato-Score-Predictor/blob/main/Rotten_Tomato_Score_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Part 1**

Scrape Rotten Tomato Movie Scores

In [ ]:
import requests
import base64
import json

list_of_movie_details = {}

#function for cleaning the json data that we get from the rotten tomato website
#this is a helper function
def clean_data_for_rotten_tomatoes(title, audience_score, critics_score, after_cursor):

    #for each variable, we check that it is a string instance so there is no data
    #integrity problems, then we replace the string types to ensure that if there
    #is no problem incase there is a quote in the movie title.

    #for each of these we also have a set default so that way we can easily filter them out
    #later if there are any problems
    if isinstance(title, str):
        title = title.replace('"', "'")
    else:
        title = "Unknown Title"

    if isinstance(audience_score, str):
        audience_score = audience_score
    else:
        audience_score = "N/A"

    if isinstance(critics_score, str):
        critics_score = critics_score
    else:
       critics_score =  "N/A"
    if isinstance(after_cursor, str):
        after_cursor = after_cursor
    else:
        after_cursor = "Unknown Cursor"
    return title, audience_score, critics_score, after_cursor

def fetch_movies(limit=1000):

    #this is the base url of the api request
    base_url = "https://www.rottentomatoes.com/napi/browse/movies_at_home/"

    #this was a part of the url that would change depending on the page
    #we wanted to go from page to page, and this is what allowed us to do so
    after_cursor = base64.b64encode(b"29").decode()

    #this just indicated that this was not the last page in the list, later will be updated by the response json
    has_next_page = True
    count = 1


    # this is our loop to go on until either there are no more pages or our limit has been reached.
    while has_next_page and count < limit:


        #here we make the actual request to the API, we used tool that translated
        # cUrl request to python, but generally it was pretty straight forward
        response = requests.get(
            f"{base_url}?after={after_cursor}",
            headers={
                "accept": "*/*",
                "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Mobile Safari/537.36",
            },
        )


        #this confirms that the request made was valid
        if response.status_code == 200:

            #loads the response data, which we will be able to parse
            data = response.json()


            #there was a lot of unsertainty about ho well this all would work, so we put a lot
            #of safegaurds regarding what the data might have (use .get() instead of direct access
            # and have the fillers of {} or [] incase they didnt exist)
            movies = data.get("grid", {}).get("list", [])
            for movie in movies:
                title, audience_score, critics_score, after_cursor = clean_data_for_rotten_tomatoes(
                    movie.get("title", "Unknown Title"),
                    movie.get("audienceScore", {}).get("scorePercent", "N/A"),
                    movie.get("criticsScore", {}).get("scorePercent", "N/A"),
                    after_cursor
                )


                #add it to our own dictionary data set
                list_of_movie_details[title] = {
                    "audience_score": audience_score,
                    "critics_score": critics_score,
                    "after_cursor": after_cursor
                }

                # Print progress
                print(f"Movie Count: {count}")
                print(f"Title: {title}")
                print(f"Audience Score: {audience_score}")
                print(f"Critics Score: {critics_score}")
                print("-" * 50)
                count += 1


            #here we were cacheing out results incase there was a problem with the API
            #we wanted to "save" our progress as we went along
            with open("latest_movie_reviews.json", "w") as f:
                json.dump(list_of_movie_details, f, indent=4)

            #here we just got the page info that contained the has_next_page variable so
            #we knew when we reached the end.
            page_info = data.get("pageInfo", {})
            has_next_page = page_info.get("hasNextPage", False)

            #this part we needed to get some help from online sources,
            #but we found that the after_cursor simply got us to the next
            # page and is fairly standard
            after_cursor = page_info.get("endCursor", "")

        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

In [ ]:
#This function scrapes the rotten tomato page and pulls all the names of the movies
#the rotten critic score and audiance score
#it does this by making a question to the public API url that is used to load the page, and then
#read the json.

#Since we are working with a large data set it we made it so that the data was constantly dumped into a .json file
#incase anything went wrng along the way.
fetch_movies(limit=10) #limited to 10 for this example

Scrape Youtube Comments from Movie Trailers

In [ ]:
from googleapiclient.discovery import build
import re
from textblob import TextBlob

class Sentiment:
    def __init__(self):
        self.count = 0

    #this is used to generate a sentiment analysis
    #we put it in a class to make the code cleaner, and
    #so that way we can track of errors without distrupting flow
    def analyze_sentiment(self, text):
        try:
            analysis = TextBlob(text)
            return analysis.sentiment.polarity
        except:
            self.count += 1
            print("problem happened")
            return 0

    #returns error count
    def countBad(self):
        print(self.count)

class YouTube():
    def __init__(self, APIKEY):

        #the following are simply used to activate our YouTube connectionss
        self.API_KEY = APIKEY
        self.YOUTUBE_API_SERVICE_NAME = 'youtube'
        self.YOUTUBE_API_VERSION = 'v3'
        self.youtube = build(self.YOUTUBE_API_SERVICE_NAME, self.YOUTUBE_API_VERSION, developerKey=self.API_KEY)


    def clean_comment(self,x):
        #returns x if x is none
        if x is None:
            return x
        #cleans the code so that way only characters we want are in there
        x = re.sub(r"[^a-zA-Z0-9\s.,!?\'\"-]", '', x)
        #finally we also added a check so that there is no HTML code in there which we found to be problematic later
        return re.sub(r'<.*?>', '', x)


    #this function is used for getting a list of video comments based on a video_id
    def get_video_comments(self, video_id, max_results=10):
        comments = []
        next_page_token = None

        while True:

            #this builds a request to youtube's commentThreads API  and fetches the top comments
            request = self.youtube.commentThreads().list(
                part="snippet",
                #sets the video ID
                videoId=video_id,
                # API allows max of 100 per page
                maxResults=min(max_results, 100),
                #similar to the rotten tomato request, this keeps track if we need to move to next page
                pageToken=next_page_token

            )

            #this makes the request and returns the json data
            response = request.execute()

            #here we do some basic parsing, the json data is nested so we need to pull only what we need out of it
            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                #we then append it to a list of comments and clean it as well.
                comments.append(self.clean_comment(comment))

            # get next page or break if no more pages
            next_page_token = response.get('nextPageToken')
            if not next_page_token or len(comments) >= max_results:
                break

        return comments


    def get_channel_id(self, channel_name):

        #here we take advantage of youtubes search function
        #we pass in the channel name, type and number of results (which is
        # just a techincial thing we must include)
        request = self.youtube.search().list(
            part="snippet",
            q=channel_name,
            type="channel",
            maxResults=1
        )
        response = request.execute()

        #same as before, we parse through the data to extract only what we really need and return an
        #error if it is not there, in this case, the channel does not exisit
        if 'items' in response and response['items']:
            return response['items'][0]['id']['channelId']
        else:
            raise ValueError(f"Channel '{channel_name}' not found.")

    def get_video_names_and_ids(self, channel_name, max_results=50):

        videos = []
        next_page_token = None
        #we begin by using one of the other functions to get the channel ID
        channel_id = self.get_channel_id(channel_name)

        #then run a loop that will continue until all of the videos are retreived
        while True:
            #here we make the request to the youtube API
            request = self.youtube.search().list(
                part="snippet",
                channelId=channel_id,
                maxResults=min(max_results, 50),  # API allows up to 50 per request
                pageToken=next_page_token,
                type="video"
            )
            response = request.execute()

            #sort through the items to get only what we need, then append it to the list
            for item in response.get('items', []):
                video_id = item['id'].get('videoId')
                video_title = item['snippet']['title']
                if video_id:
                    videos.append((video_id, video_title))

            next_page_token = response.get('nextPageToken')
            if not next_page_token or len(videos) >= max_results:
                break

        return videos

In [ ]:
import json

#Here we initilized the youtube class so we can interact with the API
youtubeAdmin = YouTube(APIKEY)

#Here we selected which youtube channels we wanted to grab the trailers from
list_of_channels = ["WarnerBrosPictures", "20thCenturyStudios", "marvel", "AppleTV", "A24", "Showtime","Netflix"]


#Here we filter through all videos that are movie trailers and store their title and youtubeID
#by analyzing the youtube video title format we saw that most trailers included the word "trailer"
#so we filtered for that, while we didnt catch ALL trailers, this was an easy quick filter

#We also grabbed the movie name so that way we can do manual checks later on
list_of_movies = []
for channel_name in list_of_channels:
    movies_in_channel = youtubeAdmin.get_video_names_and_ids(channel_name)
    for id, title in movies_in_channel:
        if "trailer" in title.lower():
            # print(title)
            list_of_movies.append((id, title.lower()))

In [ ]:
import json
#open up the rotten tomato data and load as python json object
#Remember that this was the data that we grabbed from before
with open("latest_movie_reviews.json", "r") as f:
    rt_scores = json.loads(f.read())

#find matches between the two data sets
#we generated the list of ignore_set by looking at the data that we had and found that there were movie title that were
#fairly generic and would be better off if we ignored them as to not cause any problems
#since our youtube trailer set included aditional words aside from just the movie name
ignore_set = ['Life','Gold','Heat', 'Go', 'Us', 'Pi', 'Steel', 'Ali', 'Plane', 'Man of Steel', 'App', 'Logan', 'Room', 'The Killing', 'War', 'IF', 'Moon', 'Heretic', 'The Zone of Interest', 'Ma', 'Red', 'RV', 'It', 'AfrAId', 'Ted', 'The Many Saints of Newark', 'Dick', 'Bird', 'Ran', 'Up', 'The Killing of a Sacred Deer', 'Beau Is Afraid', 'Old', 'Kill', 'Black Widow', 'IO', 'Bohemian Rhapsody', 'Her', 'Murder on the Orient Express', 'Civil War', 'After']
list_of_matches = []
title_count = {}

#we then itterated through all of the rt scores
for rt_movie_title in rt_scores:

    #checked if the title was valid, and that it is not in the ignore set
    if len(rt_movie_title) > 1 and rt_movie_title not in ignore_set:

        #we then itterated through our youtube movie trailer list
        #replaced some filler words that we noticed would cause problems
        #then we would standardize the string and check if we have a match
        #if there was a mach we would append it to a list for use later on
        for movie in list_of_movies:
            movie_title = movie[1]
            movie_title = movie_title.replace("marvel studios", "")
            movie_title = movie_title.replace("final trailer", "")

            if rt_movie_title.lower() in movie_title:
                list_of_matches.append((movie[0], rt_movie_title, rt_scores[rt_movie_title]))


#at this point we noticed that there was still inconsitancy in the data
#since the data set contained ~200 movies by this point, it was failry easy to
#manually do some cleanying and confirm that the movies we had matched up
#were truely the correct matches

Create CSV

In [ ]:
import csv
import pandas as pd

#for simple storage, here we have our list of good matches.
good_matches = [('iZwykQK9aZo', 'Blitz', {'audience_score': '36%', 'critics_score': '48%', 'after_cursor': 'MjA2OQ=='}), ('_9CmC5Rmsdw', 'A Different Man', {'audience_score': '80%', 'critics_score': '92%', 'after_cursor': 'NTk='}), ('mqqft2x_Aa4', 'The Batman', {'audience_score': '87%', 'critics_score': '85%', 'after_cursor': 'ODk='}), ('kymDzCgPwj0', 'I Saw the TV Glow', {'audience_score': '71%', 'critics_score': '84%', 'after_cursor': 'ODk='}), ('D30r0CwtIKc', 'The Whale', {'audience_score': '84%', 'critics_score': '86%', 'after_cursor': 'NzE2OQ=='}), ('sw7RElt-SvE', 'Paddington 2', {'audience_score': '88%', 'critics_score': '99%', 'after_cursor': 'MTE5'}), ('1HZAnkxdYuA', 'The Little Things', {'audience_score': '67%', 'critics_score': '44%', 'after_cursor': 'MTE5'}), ('G9jOaggGPKQ', 'Aftersun', {'audience_score': '81%', 'critics_score': '95%', 'after_cursor': 'MTQ5'}), ('EG0si5bSd6I', 'Killers of the Flower Moon', {'audience_score': '84%', 'critics_score': '93%', 'after_cursor': 'MTc5'}), ('X5oqpxi3U7M', 'The Instigators', {'audience_score': '54%', 'critics_score': '40%', 'after_cursor': 'MjA5'}), ('aLAKJu9aJys', 'Talk to Me', {'audience_score': '82%', 'critics_score': '94%', 'after_cursor': 'MjM5'}), ('5wfrDhgUMGI', 'Hidden Figures', {'audience_score': '93%', 'critics_score': '93%', 'after_cursor': 'MjY5'}), ('V6wWKNij_1M', 'Hereditary', {'audience_score': '71%', 'critics_score': '90%', 'after_cursor': 'Mjk5'}), ('x_me3xsvDgk', 'Eternals', {'audience_score': '77%', 'critics_score': '47%', 'after_cursor': 'Mjk5'}), ('cNi_HC839Wo', 'Lady Bird', {'audience_score': '79%', 'critics_score': '99%', 'after_cursor': 'Mzg5'}), ('LdOM0x0XDMo', 'Tenet', {'audience_score': '76%', 'critics_score': '70%', 'after_cursor': 'Mzg5'}), ('9ix7TUGVYIo', 'The Matrix Resurrections', {'audience_score': '63%', 'critics_score': '63%', 'after_cursor': 'NDQ5'}), ('aWzlQ2N6qqg', 'Doctor Strange in the Multiverse of Madness', {'audience_score': '85%', 'critics_score': '74%', 'after_cursor': 'NDQ5'}), ('8KVsaoveTbw', 'The Iron Claw', {'audience_score': '94%', 'critics_score': '89%', 'after_cursor': 'NDc5'}), ('43NWzay3W4s', 'Captain America: Civil War', {'audience_score': '89%', 'critics_score': '90%', 'after_cursor': 'NjI5'}), ('qvqyBWCN39o', 'Tuesday', {'audience_score': '50%', 'critics_score': '76%', 'after_cursor': 'NjU5'}), ('20GWk5cWPBs', 'You Hurt My Feelings', {'audience_score': '64%', 'critics_score': '94%', 'after_cursor': 'NjU5'}), ('U0CL-ZSuCrQ', 'In the Heights', {'audience_score': '94%', 'critics_score': '94%', 'after_cursor': 'NzE5'}), ('fKHVHzCGmF0', 'Sharper', {'audience_score': '73%', 'critics_score': '68%', 'after_cursor': 'ODM5'}), ('0pmfrE1YL4I', 'CODA', {'audience_score': '91%', 'critics_score': '94%', 'after_cursor': 'ODY5'}), ('zyYgDtY2AMY', 'Ford v Ferrari', {'audience_score': '98%', 'critics_score': '92%', 'after_cursor': 'ODY5'}), ('R_F-lVhSfx8', 'Black Mass', {'audience_score': '68%', 'critics_score': '73%', 'after_cursor': 'OTU5'}), ('ByehYal_cCs', 'Cloud Atlas', {'audience_score': '66%', 'critics_score': '66%', 'after_cursor': 'MTEwOQ=='}), ('nSbzyEJ8X9E', 'A Star Is Born', {'audience_score': '81%', 'critics_score': '98%', 'after_cursor': 'ODYwOQ=='}), ('eMQt_V2eCRY', 'The Front Room', {'audience_score': '42%', 'critics_score': '42%', 'after_cursor': 'MTE2OQ=='}), ('A5GJLwWiYSg', 'West Side Story', {'audience_score': '84%', 'critics_score': '92%', 'after_cursor': 'NjQxOQ=='}), ('DBWk6BohVXk', 'Priscilla', {'audience_score': '63%', 'critics_score': '84%', 'after_cursor': 'MTIyOQ=='}), ('5sEaYB4rLFQ', 'Fantastic Beasts: The Crimes of Grindelwald', {'audience_score': '53%', 'critics_score': '36%', 'after_cursor': 'MTI1OQ=='}), ('GLs2xxM0e78', 'Amsterdam', {'audience_score': '62%', 'critics_score': '32%', 'after_cursor': 'MTI4OQ=='}), ('LTNZmOJxuAc', 'The Lobster', {'audience_score': '65%', 'critics_score': '88%', 'after_cursor': 'MTMxOQ=='}), ('ZSzeFFsKEt4', 'The Hobbit: The Battle of the Five Armies', {'audience_score': '74%', 'critics_score': '59%', 'after_cursor': 'MTM0OQ=='}), ('ptqe7s6pO7g', 'The Tragedy of Macbeth', {'audience_score': '74%', 'critics_score': '92%', 'after_cursor': 'MTM3OQ=='}), ('6Nxc-3WpMbg', 'Kingsman: The Golden Circle', {'audience_score': '64%', 'critics_score': '51%', 'after_cursor': 'MTQwOQ=='}), ('gSMxBLlA8qY', 'Richard Jewell', {'audience_score': '96%', 'critics_score': '77%', 'after_cursor': 'MTQzOQ=='}), ('Dp2ufFO4QGg', 'The Art of Racing in the Rain', {'audience_score': '96%', 'critics_score': '44%', 'after_cursor': 'MTQzOQ=='}), ('N_yfmHCkSB0', 'Boston Strangler', {'audience_score': '68%', 'critics_score': '68%', 'after_cursor': 'MTU1OQ=='}), ('0dCfbBwFI2Y', 'The Spectacular Now', {'audience_score': '76%', 'critics_score': '92%', 'after_cursor': 'MTU4OQ=='}), ('n0UFgbGZk10', 'Problemista', {'audience_score': '85%', 'critics_score': '86%', 'after_cursor': 'MTYxOQ=='}), ('nvP-ZrzA2lk', 'Lincoln', {'audience_score': '80%', 'critics_score': '90%', 'after_cursor': 'MTc2OQ=='}), ('j307q8zMD3Y', 'Causeway', {'audience_score': '72%', 'critics_score': '85%', 'after_cursor': 'MTgyOQ=='}), ('GVQbeG5yW78', 'Just Mercy', {'audience_score': '99%', 'critics_score': '85%', 'after_cursor': 'MTg1OQ=='}), ('kjC1zmZo30U', 'Tag', {'audience_score': '57%', 'critics_score': '55%', 'after_cursor': 'MTg1OQ=='}), ('Kwp32zLc08c', 'After Yang', {'audience_score': '68%', 'critics_score': '89%', 'after_cursor': 'MTk3OQ=='}), ('ZBvK6ni97W8', 'The Judge', {'audience_score': '72%', 'critics_score': '49%', 'after_cursor': 'MjMwOQ=='}), ('tN8o_E_f9FQ', 'The Darkest Minds', {'audience_score': '71%', 'critics_score': '15%', 'after_cursor': 'MjM5OQ=='}), ('sSjtGqRXQ9Y', 'Judas and the Black Messiah', {'audience_score': '95%', 'critics_score': '96%', 'after_cursor': 'MjgxOQ=='}), ('mjKEXxO2KNE', 'Sully', {'audience_score': '84%', 'critics_score': '85%', 'after_cursor': 'MjgxOQ=='}), ('w9Rx6-GaSIE', 'Mid90s', {'audience_score': '81%', 'critics_score': '81%', 'after_cursor': 'MjkzOQ=='}), ('fH0cEP0mvlU', 'This Is Where I Leave You', {'audience_score': '60%', 'critics_score': '44%', 'after_cursor': 'Mjk5OQ=='}), ('3MM8OkVT0hw', 'The Hate U Give', {'audience_score': '81%', 'critics_score': '97%', 'after_cursor': 'MzM1OQ=='}), ('m5LmfARzwDU', 'Shallow Hal', {'audience_score': '45%', 'critics_score': '49%', 'after_cursor': 'MzU2OQ=='}), ('-rjMwSTeVeo', 'Stop Making Sense', {'audience_score': '97%', 'critics_score': '100%', 'after_cursor': 'MzgwOQ=='}), ('1O3iRdiplB0', 'The Heat', {'audience_score': '71%', 'critics_score': '65%', 'after_cursor': 'MzkyOQ=='}), ('IrabKK9Bhds', 'They Shall Not Grow Old', {'audience_score': '91%', 'critics_score': '99%', 'after_cursor': 'Mzk1OQ=='}), ('DmmHvnS0IKM', 'Blinded by the Light', {'audience_score': '91%', 'critics_score': '89%', 'after_cursor': 'NDA0OQ=='}), ('Af77C4zUkjs', 'Come From Away', {'audience_score': '95%', 'critics_score': '98%', 'after_cursor': 'NDEwOQ=='}), ('qOYrpk-A45s', 'Occupied City', {'audience_score': '75%', 'critics_score': '72%', 'after_cursor': 'NDEwOQ=='}), ('UIMYNVkZBSo', 'Hall Pass', {'audience_score': '40%', 'critics_score': '33%', 'after_cursor': 'NDE5OQ=='}), ('OOPHFQZ5aiM', 'Water for Elephants', {'audience_score': '70%', 'critics_score': '60%', 'after_cursor': 'NDI1OQ=='}), ('VpUeQV8sdOc', 'Horrible Bosses', {'audience_score': '70%', 'critics_score': '69%', 'after_cursor': 'NDI1OQ=='}), ('6JnFaltqnAY', '20th Century Women', {'audience_score': '74%', 'critics_score': '88%', 'after_cursor': 'NDYxOQ=='}), ('NcE2fj-EtJY', 'The Bling Ring', {'audience_score': '33%', 'critics_score': '60%', 'after_cursor': 'NDY3OQ=='}), ('hyzQjVUmIxk', 'Eddie the Eagle', {'audience_score': '82%', 'critics_score': '82%', 'after_cursor': 'NTQyOQ=='}), ('lD41XdWcmbY', 'Shaft', {'audience_score': '68%', 'critics_score': '88%', 'after_cursor': 'ODYzOQ=='}), ('6EJGnU2AmV4', 'Close', {'audience_score': '35%', 'critics_score': '35%', 'after_cursor': 'NTY2OQ=='}), ('KBDE4uznmIw', 'Bride Wars', {'audience_score': '51%', 'critics_score': '10%', 'after_cursor': 'NTkzOQ=='}), ('go1jaIRQc-o', 'Breakthrough', {'audience_score': '81%', 'critics_score': '60%', 'after_cursor': 'NTk2OQ=='}), ('5hJR8hEsLZU', 'The Eternal Daughter', {'audience_score': '46%', 'critics_score': '95%', 'after_cursor': 'NjA4OQ=='}), ('JjysgllBzHQ', 'Life After Beth', {'audience_score': '31%', 'critics_score': '46%', 'after_cursor': 'NjIwOQ=='}), ('i5l6a5RiR1E', 'Stuber', {'audience_score': '79%', 'critics_score': '43%', 'after_cursor': 'NjIwOQ=='}), ('IZUrhfCl0Xc', 'Bad Sister', {'audience_score': '56%', 'critics_score': '', 'after_cursor': 'NjYyOQ=='}), ('dGBe8xsWGlo', 'The Death of Dick Long', {'audience_score': '86%', 'critics_score': '75%', 'after_cursor': 'NzE5OQ=='}), ('6tC1yOUvvMo', 'Jersey Boys', {'audience_score': '62%', 'critics_score': '51%', 'after_cursor': 'NzQwOQ=='}), ('nPfYXXg65qA', 'Keeping Up With the Joneses', {'audience_score': '37%', 'critics_score': '20%', 'after_cursor': 'Nzc2OQ=='}), ('VP5FW5KA6Go', 'Ramona and Beezus', {'audience_score': '71%', 'critics_score': '71%', 'after_cursor': 'Nzc5OQ=='}), ('ld4eE2HU-ig', 'The Exception', {'audience_score': '67%', 'critics_score': '75%', 'after_cursor': 'ODE4OQ=='}), ('6pwnwnzk8L8', 'Chasing Mavericks', {'audience_score': '70%', 'critics_score': '32%', 'after_cursor': 'ODc4OQ=='})]

#once again start an instance of our youtube class but now also an instance of the sentimet class for analysis.
youtubeAdmin = YouTube(APIKEY)
sentimetAdmin = Sentiment()


#here things get a little funky, when trying to complete this part using standard pandas approach,
#I found that the data would get mixed up and disorded. This is likely due to different languages, emojies,
#and odd characters, however we were unable to easily resolve it.

#instead since I had some experiance working with the csv library, I decided to write a script that would generate
#the csv directly.

#here we open the file,
with open('movies_comments_cleanTest_with_sentimate.csv', mode='w', newline='', encoding='utf-8') as file:

    #initialize our write
    writer = csv.writer(file)

    #define what our column headers would be, here are the core 4, however we will have many more generated below
    headers = ['trailer_id', 'movie_name', 'audience_score', 'critics_score']

    #this will generate our comment columns (1-100)
    for i in range(1, 101):
        headers.append(f'comment_{i}')

    #this will gen our sentiment (1-100)
    for i in range(1, 101):
        headers.append(f'comment_sentiment_{i}')

    #this will write out column row
    writer.writerow(headers)

    #counter here is just used to keep track of progress
    count = 0

    #here we through every match we have
    #get the youtube comments from the API
    #pass them through the sentiment function
    #then write all this data into the corrilating columns
    for i in good_matches:

        count += 1
        trailer_id = i[0]
        movie_name = i[1]
        audience_score = i[2]["audience_score"]
        critics_score = i[2]["critics_score"]
        try:

            comments = youtubeAdmin.get_video_comments(trailer_id, max_results=100)

        except:
            print(f"Youtube Problem Comments for {movie_name}: {len(comments)}")
            comments = []

        try:
            comments_sentimate = [sentimetAdmin.analyze_sentiment(x) for x in comments]
        except:
            print(f"Sentimate for {movie_name}: {len(comments_sentimate)}")
            comments_sentimate = []
        # Create row
        row = [trailer_id, movie_name, audience_score, critics_score] + comments + comments_sentimate
        writer.writerow(row)
        print(f"Success count: {count}")

#at this point the data is all ready for analysis

In [ ]:
#check that all worked fine
sentimetAdmin.countBad()

**Part 2**

Load, Validate, and Split the Data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Load CSV
df = pd.read_csv('movies_comments_cleanTest_with_sentimate.csv', encoding='utf-8', on_bad_lines='skip',  engine='python')

# Define valid columns
valid_columns = [f'comment_sentiment_{i}' for i in range(1, 101)]

# Drop invalid rows(rows that dont have values in all the comment sentiment columns)
df_valid = df.dropna(subset=valid_columns).copy()

# Drop rows with over 43 columns of sentiment equal to 0
df_valid["zero_count"] = (df_valid[valid_columns] == 0).sum(axis=1)
df_valid_reduced = df_valid[df_valid["zero_count"] <= 43].drop("zero_count", axis=1)

# Ensure sentiment scores are numerical, if not they will become NaN
for col in valid_columns:
    df_valid_reduced[col] = pd.to_numeric(df_valid_reduced[col])

# Assign the sentiment columns to X, filling all NaN with 0
X = df_valid_reduced[valid_columns].fillna(0)

# Assign critics_score to y (remove '%' and convert to float)
y = pd.to_numeric(df_valid_reduced['critics_score'].str.rstrip('%'))

# Drop rows where y is NaN
X = X[~y.isna()]
y = y[~y.isna()]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

print(f"Training set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")

Train Regression Model

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Computes weights inversely proportional to the frequency of scores to balance skewed distribution
def compute_weights(critic_scores):
    bins = [0, 20, 40, 60, 80, 100]
    counts, _ = np.histogram(critic_scores, bins=bins)
    weights = 1 / np.sqrt(counts + 1e-6)
    bin_indices = np.digitize(critic_scores, bins) - 1
    bin_indices = np.clip(bin_indices, 0, len(weights) - 1)
    return weights[bin_indices]

# Initialize model with regularization
model = xgb.XGBRegressor(
    learning_rate=0.1,
    n_estimators=200,
    max_depth=4,
    subsample=0.8,          # Use 80% of the data per tree
    colsample_bytree=0.8,   # Use 80% of the features per tree
    reg_alpha=0.1,          # L1 regularization term
    reg_lambda=1.0,         # L2 regularization term
    random_state=42
)

#Train model with weights
model.fit(
    X_train,
    y_train,
    sample_weight=compute_weights(y_train)
)

# Evaluate model
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Calculate percent accuracy within 10% in test set
percentage_difference_test = 100 * np.abs(y_pred - y_test) / y_test
accurate_within_10_percent_test = np.sum(percentage_difference_test <= 10)
accuracy_percentage_test = 100 * accurate_within_10_percent_test / len(y_test)
# Calculate percent accuracy within 5% in test set
accurate_within_5_percent_test = np.sum(percentage_difference_test <= 5)
accuracy_percentage_5_test = 100 * accurate_within_5_percent_test / len(y_test)

# Calculate percent accuracy within 10% in full dataset
y_pred_all = model.predict(X)
percentage_difference_test_all = 100 * np.abs(y_pred_all - y) / y
accurate_within_10_percent_test_all = np.sum(percentage_difference_test_all <= 10)
accuracy_percentage_test_all = 100 * accurate_within_10_percent_test_all / len(y)
# Calculate percent accuracy within 5% in full dataset
accurate_within_5_percent_test_all = np.sum(percentage_difference_test_all <= 5)
accuracy_percentage_5_test_all = 100 * accurate_within_5_percent_test_all / len(y)

# Print results
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}\n")
print(f"Percent accuracy within 10% in test set: {accuracy_percentage_test:.2f}%")
print(f"Percent accuracy within 10% in full dataset: {accuracy_percentage_test_all:.2f}%\n")
print(f"Percent accuracy within 5% in test set: {accuracy_percentage_5_test:.2f}%")
print(f"Percent accuracy within 5% in full dataset: {accuracy_percentage_5_test_all:.2f}%")

Store Predicted Scores in an SQL Database

In [3]:
import sqlite3

conn = sqlite3.connect('movies_scores.db')
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS PredictedScores (
    MovieID INTEGER,
    Title TEXT,
    AudienceScore REAL,
    CriticScore REAL,
    PredictedScore REAL
)
''')

# Clear the table if it exists
cursor.execute('DELETE FROM PredictedScores')

for i, row in df_valid.iterrows():
    try:
        # Predict scores for each row
        features = pd.DataFrame([row.loc[X.columns].values], columns=X.columns)
        predicted_score = float(model.predict(features)[0])

        # Insert into the SQL table
        cursor.execute('''
        INSERT INTO PredictedScores (MovieID, Title, AudienceScore, CriticScore, PredictedScore)
        VALUES (?, ?, ?, ?, ?)
        ''', (row['trailer_id'], row['movie_name'], row['audience_score'], row['critics_score'], predicted_score))
    except KeyError as e:
        print(f"Skipping row {i} due to missing key: {e}")
    except Exception as e:
        print(f"Skipping row {i} due to error: {e}")

conn.commit()

In [ ]:
# Print the table
query = 'SELECT AudienceScore, CriticScore, PredictedScore FROM PredictedScores'
df = pd.read_sql_query(query, conn)
print(df)

conn.close()

Visualize Data and Model Performance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to db
conn = sqlite3.connect('movies_scores.db')
query = 'SELECT * FROM PredictedScores'
df = pd.read_sql_query(query, conn)

# Ensure data is numeric
df['AudienceScore'] = pd.to_numeric(df['AudienceScore'].str.rstrip('%'))
df['CriticScore'] = pd.to_numeric(df['CriticScore'].str.rstrip('%'))
df['PredictedScore'] = pd.to_numeric(df['PredictedScore'])

# Visualization 1 - Scatter Plot
plt.figure(figsize=(10, 6))
plt.scatter(df['CriticScore'], df['PredictedScore'], label='Critics vs. Model', alpha=0.7, color='blue')
plt.scatter(df['AudienceScore'], df['PredictedScore'], label='Audience vs. Model', alpha=0.7, color='green')
plt.plot([30, 100], [30, 100], 'r--', label='Perfect Prediction', alpha=0.8)
plt.title("Model Predictions vs Critics' and Audience Scores")
plt.xlabel("Actual Scores (%)")
plt.ylabel("Model Predictions (%)")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Visualization 2 - Histogram of Residuals
plt.figure(figsize=(10, 6))
critics_residuals = df['PredictedScore'] - df['CriticScore']
audience_residuals = df['PredictedScore'] - df['AudienceScore']
plt.hist(critics_residuals, bins=20, alpha=0.7, label='Critics Residuals', color='blue')
plt.hist(audience_residuals, bins=20, alpha=0.7, label='Audience Residuals', color='green')
plt.axvline(0, color='red', linestyle='dashed', linewidth=1, label='Zero Residual')
plt.title("Residuals of Model Predictions")
plt.xlabel("Residual Value (Model - Actual)")
plt.ylabel("Frequency")
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# Visualization 3 - Boxplot
plt.figure(figsize=(10, 6))
boxplot_data = [df['CriticScore'], df['AudienceScore'], df['PredictedScore']]
plt.boxplot(boxplot_data, vert=False, patch_artist=True, labels=['Critics', 'Audience', 'Model'])
plt.title("Distribution of Critics, Audience, and Model Scores")
plt.xlabel("Score (%)")
plt.grid(alpha=0.3)
plt.show()

# Visualization 4 - Score Bar Chart
plt.figure(figsize=(15, 8))
bar_width = 0.25
indices = range(len(df))
plt.bar([i - bar_width for i in indices], df['CriticScore'], width=bar_width, label='Critics', color='blue')
plt.bar(indices, df['PredictedScore'], width=bar_width, label='Predicted', color='orange')
plt.bar([i + bar_width for i in indices], df['AudienceScore'], width=bar_width, label='Audience', color='green')
plt.title("Critics, Predicted, and Audience Scores for Each Movie", fontsize=16)
plt.xlabel("Movies", fontsize=14)
plt.ylabel("Scores (%)", fontsize=14)
plt.xticks(indices, df['Title'], rotation=45, ha='right')  # Movie titles on x-axis
plt.legend()
plt.tight_layout()
plt.grid(axis='y', alpha=0.3)
plt.show()

# Visualization 5 - Error Bar Chart
plt.figure(figsize=(15, 8))
audience_error = abs(df['PredictedScore'] - df['AudienceScore'])
critic_error = abs(df['PredictedScore'] - df['CriticScore'])
bar_width = 0.35
indices = range(len(df))
plt.bar([i - bar_width/2 for i in indices], critic_error, width=bar_width, label='Critics Error', color='blue')
plt.bar([i + bar_width/2 for i in indices], audience_error, width=bar_width, label='Audience Error', color='green')
plt.title("Absolute Errors of Predicted Scores", fontsize=16)
plt.xlabel("Movies", fontsize=14)
plt.ylabel("Absolute Error (%)", fontsize=14)
plt.xticks(indices, df['Title'], rotation=45, ha='right')  # Movie titles on x-axis
plt.legend()
plt.tight_layout()
plt.grid(axis='y', alpha=0.3)
plt.show()